In [1]:
import pandas as pd
import numpy as np

df_s = pd.read_excel("shots.xlsx")

In [2]:
df_s.shape

(31736, 77)

In [3]:
df_s.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'id', 'index', 'period', 'timestamp',
       'minute', 'second', 'possession', 'possession_team_name', 'player_id',
       'player_name', 'duration', 'under_pressure', 'play_pattern_name',
       'shot_freeze_frame', 'shot_type_name', 'shot_technique_name',
       'shot_outcome_name', 'shot_body_part_name', 'shot_statsbomb_xg',
       'previous_type_name', 'previous_pass_height_name', 'location_x',
       'location_y', 'shot_end_location_x', 'shot_end_location_y', 'match_id',
       'home_team_home_team_name', 'away_team_away_team_name', 'home_away',
       'pack_density', 'league_trophy', 'location0', 'location1', 'location2',
       'location3', 'location4', 'location5', 'location6', 'location7',
       'location8', 'location9', 'location10', 'location11', 'location12',
       'location13', 'location14', 'location15', 'location16', 'location17',
       'location18', 'location19', 'location', 'inside0', 'inside1', 'inside2',
       'inside3', 'insi

In [4]:
unnecessary_columns = ['Unnamed: 0', 'Unnamed: 0.1', 'location0', 'location1', 'location2',
       'location3', 'location4', 'location5', 'location6', 'location7',
       'location8', 'location9', 'location10', 'location11', 'location12',
       'location13', 'location14', 'location15', 'location16', 'location17',
       'location18', 'location19', 'location', 'inside0', 'inside1', 'inside2',
       'inside3', 'inside4', 'inside5', 'inside6', 'inside7', 'inside8',
       'inside9', 'inside10', 'inside11', 'inside12', 'inside13', 'inside14',
       'inside15', 'inside16', 'inside17', 'inside18', 'inside19', 'gk_pos_x',
       'gk_pos_y']

df_s = df_s.drop(unnecessary_columns, axis = 1)

In [5]:
df_openplay = df_s.loc[df_s['shot_type_name'] == 'Open Play']

In [6]:
df = df_openplay.copy()

In [7]:
df['shot_outcome_name'].value_counts()

Off T               9498
Saved               7162
Blocked             7148
Goal                3377
Wayward             1467
Post                 657
Saved Off Target     147
Saved to Post         73
Name: shot_outcome_name, dtype: int64

In [8]:
df['shot_outcome'] = np.where((df['shot_outcome_name'] == 'Goal'), 
                                         'goal', 'No_goal')
df['shot_outcome_name'].value_counts()

Off T               9498
Saved               7162
Blocked             7148
Goal                3377
Wayward             1467
Post                 657
Saved Off Target     147
Saved to Post         73
Name: shot_outcome_name, dtype: int64

In [9]:
df['shot_distance']=round(np.sqrt((df['shot_end_location_x']-df['location_x'])**2+(df['shot_end_location_y']-df['location_y'])**2),1)
df['shot_a'] = np.sqrt((df['location_x']-120)**2+(df['location_y']-36)**2)
df['shot_b']=np.sqrt((df['location_x']-120)**2+(df['location_y']-44)**2)

df['shot_a'] = df['shot_a'].round(2)
df['shot_b'] = df['shot_b'].round(2)


df['shot_angle']=np.arccos((df['shot_a']**2+df['shot_b']**2-64)/(2*df['shot_a']*df['shot_b']))#calculating angle using cosine rule
df['shot_angle']=np.degrees(df['shot_angle'])
df['shot_angle']=round(df['shot_angle'],2)

/Users/aritramajumdar/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [10]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['play_pattern_name'] = labelencoder.fit_transform(df['play_pattern_name'])
df['shot_type_name'] = labelencoder.fit_transform(df['shot_type_name'])
df['shot_technique_name'] = labelencoder.fit_transform(df['shot_technique_name'])
df['shot_body_part_name'] = labelencoder.fit_transform(df['shot_body_part_name'])
df['previous_type_name'] = labelencoder.fit_transform(df['previous_type_name'])
df['previous_pass_height_name'] = labelencoder.fit_transform(df['previous_pass_height_name'])
df['home_away'] = labelencoder.fit_transform(df['home_away'])

v = ['shot_outcome_name']
df = df.drop(v, axis = 1)

df['shot_outcome'] = labelencoder.fit_transform(df['shot_outcome'])
print(df['shot_outcome'].value_counts())

0    26152
1     3377
Name: shot_outcome, dtype: int64


In [11]:
df = df[['shot_technique_name',
        'shot_body_part_name', 'location_x', 'location_y',
        'shot_distance', 'shot_a', 'shot_b',
       'shot_angle', 'shot_statsbomb_xg', 'shot_outcome']]

df = df.dropna()
print(df['shot_outcome'].value_counts())

0    26151
1     3377
Name: shot_outcome, dtype: int64


In [12]:
variables = ['shot_technique_name',
        'shot_body_part_name', 'location_x', 'location_y',
        'shot_distance', 'shot_a', 'shot_b',
       'shot_angle', 'shot_statsbomb_xg']

target = 'shot_outcome'

X = df[variables]
y = df[target]


print(X.shape)
print(y.shape)

(29528, 9)
(29528,)


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.10, 
                                                    stratify = y)
g = ['shot_statsbomb_xg']

X_train = X_train.drop(g, axis = 1)

z = X_test[['shot_statsbomb_xg']]

X_test = X_test.drop(g, axis = 1)


sm = SMOTE(random_state=42, kind = 'borderline1')

X_resampled, Y_resampled = sm.fit_resample(X_train, y_train)

/Users/aritramajumdar/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
/Users/aritramajumdar/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/Users/aritramaju

In [15]:
lgb = LGBMClassifier()
lgb.fit(X_resampled, Y_resampled)
y_pred = lgb.predict(X_test.values)
accuracy_score(y_test, y_pred)

0.8642058923129021

In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.87      0.92      2615
           1       0.45      0.78      0.57       338

    accuracy                           0.86      2953
   macro avg       0.71      0.83      0.74      2953
weighted avg       0.91      0.86      0.88      2953



In [17]:
print(confusion_matrix(y_test, y_pred))

[[2288  327]
 [  74  264]]


In [18]:
import eli5


variables = ['shot_technique_name',
        'shot_body_part_name', 'location_x', 'location_y',
        'shot_distance', 'shot_a', 'shot_b',
       'shot_angle']

eli5.show_weights(lgb, 
                  feature_names=variables)

/Users/aritramajumdar/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/aritramajumdar/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.3265,shot_distance
0.2045,shot_angle
0.1180,shot_body_part_name
0.1073,location_x
0.1024,shot_technique_name
0.0634,shot_b
0.0533,shot_a
0.0247,location_y


In [19]:
y_test = pd.DataFrame(y_test)
y_pred_proba = lgb.predict_proba(X_test.values)
df_test_final = pd.concat([X_test, z, y_test], axis = 1)
df_test_final = df_test_final.reset_index()
t = pd.DataFrame(y_pred_proba, columns = ['ng', 'g'])
shots_test = pd.concat([df_test_final, t], axis = 1)
shots_test['shot_statsbomb_xg'] = shots_test['shot_statsbomb_xg'].round(2)
compare_xg = shots_test[['shot_statsbomb_xg', 'g', 'shot_outcome']]
cor_df = compare_xg.corr()

In [20]:
cor_df

,shot_statsbomb_xg,g,shot_outcome
shot_statsbomb_xg,1.000000,0.527058,0.435963
g,0.527058,1.000000,0.560028
shot_outcome,0.435963,0.560028,1.000000


In [ ]:
from joblib import Parallel, delayed 
import joblib
joblib.dump(lgb, 'statsbomb_all_xg_model_openplay.pkl')